<a href="https://colab.research.google.com/github/Subhr74/Git-files/blob/master/aadhar_data_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install polars pyarrow --quiet

import polars as pl
import os


In [3]:
files = [
    "api_data_aadhar_biometric_0_500000.csv",
    "api_data_aadhar_biometric_500000_1000000.csv",
    "api_data_aadhar_biometric_1000000_1500000.csv",
    "api_data_aadhar_biometric_1500000_1861108.csv",
    # add the remaining 8 chunks here, e.g.
    "api_data_aadhar_demographic_0_500000.csv",
    "api_data_aadhar_demographic_500000_1000000.csv",
    "api_data_aadhar_demographic_1000000_1500000.csv",
    "api_data_aadhar_demographic_1500000_2000000.csv",
    "api_data_aadhar_demographic_2000000_2071700.csv",
    "api_data_aadhar_enrolment_0_500000.csv",
    "api_data_aadhar_enrolment_500000_1000000.csv",
    "api_data_aadhar_enrolment_1000000_1006029.csv",  # replace with actual name
]


In [4]:
for f in files:
    print(f, "exists:", os.path.exists(f))


api_data_aadhar_biometric_0_500000.csv exists: True
api_data_aadhar_biometric_500000_1000000.csv exists: True
api_data_aadhar_biometric_1000000_1500000.csv exists: True
api_data_aadhar_biometric_1500000_1861108.csv exists: True
api_data_aadhar_demographic_0_500000.csv exists: True
api_data_aadhar_demographic_500000_1000000.csv exists: True
api_data_aadhar_demographic_1000000_1500000.csv exists: True
api_data_aadhar_demographic_1500000_2000000.csv exists: True
api_data_aadhar_demographic_2000000_2071700.csv exists: True
api_data_aadhar_enrolment_0_500000.csv exists: True
api_data_aadhar_enrolment_500000_1000000.csv exists: True
api_data_aadhar_enrolment_1000000_1006029.csv exists: True


In [5]:
files_biometric = [
    "api_data_aadhar_biometric_0_500000.csv",
    "api_data_aadhar_biometric_500000_1000000.csv",
    "api_data_aadhar_biometric_1000000_1500000.csv",
    "api_data_aadhar_biometric_1500000_1861108.csv",
]

files_demographic = [
    "api_data_aadhar_demographic_0_500000.csv",
    "api_data_aadhar_demographic_500000_1000000.csv",
    "api_data_aadhar_demographic_1000000_1500000.csv",
    "api_data_aadhar_demographic_1500000_2000000.csv",
    "api_data_aadhar_demographic_2000000_2071700.csv",
]

files_enrolment = [
    "api_data_aadhar_enrolment_0_500000.csv",
    "api_data_aadhar_enrolment_500000_1000000.csv",
    "api_data_aadhar_enrolment_1000000_1006029.csv",
]


In [7]:
import polars as pl
import os

def load_chunks(files, cols):
    dfs = []
    for f in files:
        if not os.path.exists(f):
            print("NOT FOUND:", f)
            continue
        df = pl.read_csv(
            f,
            has_header=True,
            new_columns=cols,
            infer_schema_length=1000,
            ignore_errors=True
        ).with_columns(
            pl.lit(f).alias("source_file")
        )
        dfs.append(df)
    return pl.concat(dfs, how="vertical_relaxed")


In [8]:
bio_cols   = ["date", "state", "district", "pincode", "bioage517", "bioage17"]
demo_cols  = ["date", "state", "district", "pincode", "col5", "col6"]   # TEMP names
enrol_cols = ["date", "state", "district", "pincode", "col5"]          # TEMP name

df_bio   = load_chunks(files_biometric,   bio_cols)
df_demo  = load_chunks(files_demographic, demo_cols)
df_enrol = load_chunks(files_enrolment,   enrol_cols)

df_bio.shape, df_demo.shape, df_enrol.shape


((1861108, 7), (2071700, 7), (1006029, 8))

In [11]:
def clean_common(df):
    # Ensure pincode is a 6‑digit integer
    df = df.with_columns(
        pl.col("pincode").cast(pl.Utf8).alias("pincode_str")
    ).with_columns(
        pl.when(pl.col("pincode_str").str.contains(r"^\d{6}$"))
          .then(pl.col("pincode_str").cast(pl.Int64))
          .otherwise(None)
          .alias("pincode")
    ).drop("pincode_str")

    # Convert date "dd-mm-yyyy" to date type
    df = df.with_columns(
        pl.col("date").str.to_date(format="%d-%m-%Y", strict=False)
    )

    # Keep only rows where join keys are present
    return df.drop_nulls(["date", "state", "district", "pincode"])


# apply to all three dataframes
df_bio   = clean_common(df_bio)
df_demo  = clean_common(df_demo)
df_enrol = clean_common(df_enrol)

df_bio.head(3), df_demo.head(3), df_enrol.head(3)


(shape: (3, 7)
 ┌────────────┬───────────┬──────────────┬─────────┬───────────┬──────────┬─────────────────────────┐
 │ date       ┆ state     ┆ district     ┆ pincode ┆ bioage517 ┆ bioage17 ┆ source_file             │
 │ ---        ┆ ---       ┆ ---          ┆ ---     ┆ ---       ┆ ---      ┆ ---                     │
 │ date       ┆ str       ┆ str          ┆ i64     ┆ i64       ┆ i64      ┆ str                     │
 ╞════════════╪═══════════╪══════════════╪═════════╪═══════════╪══════════╪═════════════════════════╡
 │ 2025-03-01 ┆ Haryana   ┆ Mahendragarh ┆ 123029  ┆ 280       ┆ 577      ┆ api_data_aadhar_biometr │
 │            ┆           ┆              ┆         ┆           ┆          ┆ ic_0_50…                │
 │ 2025-03-01 ┆ Bihar     ┆ Madhepura    ┆ 852121  ┆ 144       ┆ 369      ┆ api_data_aadhar_biometr │
 │            ┆           ┆              ┆         ┆           ┆          ┆ ic_0_50…                │
 │ 2025-03-01 ┆ Jammu and ┆ Punch        ┆ 185101  ┆ 643       ┆ 10

In [12]:
# Rename demographic columns
df_demo = df_demo.rename({
    "col5": "male",
    "col6": "female",
})

# Rename enrolment columns
df_enrol = df_enrol.rename({
    "col5": "enrol_total",      # total enrolments
    "age_5_17": "enrol_5_17",
    "age_18_greater": "enrol_18_plus",
})

df_demo.columns, df_enrol.columns


(['date', 'state', 'district', 'pincode', 'male', 'female', 'source_file'],
 ['date',
  'state',
  'district',
  'pincode',
  'enrol_total',
  'enrol_5_17',
  'enrol_18_plus',
  'source_file'])

In [18]:
# biometric + demographic
df_bd = df_bio.join(
    df_demo,
    on=["date", "state", "district", "pincode"],
    how="outer",          # keep this; warning is harmless
    suffix="_demo",
)

# add enrolment
df_all = df_bd.join(
    df_enrol,
    on=["date", "state", "district", "pincode"],
    how="outer",          # same here
    suffix="_enrol",
)

df_all.shape, df_all.head(5)


/tmp/ipython-input-4106248347.py:2: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  df_bd = df_bio.join(
/tmp/ipython-input-4106248347.py:10: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  df_all = df_bd.join(


((3053862, 22),
 shape: (5, 22)
 ┌────────────┬───────────┬───────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
 │ date       ┆ state     ┆ district  ┆ pincode ┆ … ┆ enrol_tot ┆ enrol_5_1 ┆ enrol_18_ ┆ source_fi │
 │ ---        ┆ ---       ┆ ---       ┆ ---     ┆   ┆ al        ┆ 7         ┆ plus      ┆ le_enrol  │
 │ date       ┆ str       ┆ str       ┆ i64     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
 │            ┆           ┆           ┆         ┆   ┆ i64       ┆ i64       ┆ i64       ┆ str       │
 ╞════════════╪═══════════╪═══════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
 │ 2025-03-01 ┆ Uttar     ┆ Gorakhpur ┆ 273213  ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
 │            ┆ Pradesh   ┆           ┆         ┆   ┆           ┆           ┆           ┆           │
 │ 2025-03-01 ┆ Andhra    ┆ Chittoor  ┆ 517132  ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
 │            ┆ Pradesh   ┆           ┆         ┆ 

In [19]:
df_all.schema


Schema([('date', Date),
        ('state', String),
        ('district', String),
        ('pincode', Int64),
        ('bioage517', Int64),
        ('bioage17', Int64),
        ('source_file', String),
        ('date_demo', Date),
        ('state_demo', String),
        ('district_demo', String),
        ('pincode_demo', Int64),
        ('male', Int64),
        ('female', Int64),
        ('source_file_demo', String),
        ('date_enrol', Date),
        ('state_enrol', String),
        ('district_enrol', String),
        ('pincode_enrol', Int64),
        ('enrol_total', Int64),
        ('enrol_5_17', Int64),
        ('enrol_18_plus', Int64),
        ('source_file_enrol', String)])

In [20]:
state_summary = df_all.group_by("state").agg([
    pl.count().alias("rows"),
    pl.col("bioage517").sum().alias("bio517_sum"),
    pl.col("bioage17").sum().alias("bio17_sum"),
    pl.col("male").sum().alias("male_sum"),
    pl.col("female").sum().alias("female_sum"),
    pl.col("enrol_total").sum().alias("enrol_sum"),
]).sort("enrol_sum", descending=True)

state_summary.head(10)


/tmp/ipython-input-997343630.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("rows"),


state,rows,bio517_sum,bio17_sum,male_sum,female_sum,enrol_sum
str,u32,i64,i64,i64,i64,i64
null,800847,0,0,470654,4661723,743244
"""Uttar Pradesh""",186977,7063316,3866521,752760,7248217,555534
"""Madhya Pradesh""",83405,3844746,3284504,391437,2391002,377317
"""Maharashtra""",181099,4051383,6544067,252490,4379568,284266
"""West Bengal""",163020,1205330,1731560,225587,3384133,278402
"""Bihar""",103750,2483622,3085304,355251,4143338,246873
"""Rajasthan""",98699,2438003,2282785,245345,2452357,226608
"""Gujarat""",107385,1662524,1974599,202043,1540232,180311
"""Tamil Nadu""",222343,2592145,2787080,291970,1735633,170137


In [21]:
district_summary = df_all.group_by(["state", "district"]).agg([
    pl.col("enrol_total").sum().alias("enrol_sum"),
    pl.col("bioage517").sum().alias("bio517_sum"),
    pl.col("bioage17").sum().alias("bio17_sum"),
]).sort("enrol_sum", descending=True).head(20)

district_summary


state,district,enrol_sum,bio517_sum,bio17_sum
str,str,i64,i64,i64
null,null,743244,0,0
"""West Bengal""","""Murshidabad""",34970,111585,159840
"""Maharashtra""","""Thane""",31803,245906,442903
"""West Bengal""","""South 24 Parganas""",27745,121398,145339
"""Maharashtra""","""Pune""",25014,329577,373112
…,…,…,…,…
"""Haryana""","""Faridabad""",17052,76673,79714
"""Maharashtra""","""Nashik""",16881,246089,421822
"""Uttar Pradesh""","""Sitapur""",16857,224608,73024


In [22]:
state_name = "West Bengal"

top_districts_state = (
    df_all
    .filter(pl.col("state") == state_name)
    .group_by("district")
    .agg([
        pl.col("enrol_total").sum().alias("enrol_sum"),
        pl.col("bioage517").sum().alias("bio517_sum"),
        pl.col("bioage17").sum().alias("bio17_sum"),
    ])
    .sort("enrol_sum", descending=True)
    .head(20)
)

top_districts_state


district,enrol_sum,bio517_sum,bio17_sum
str,i64,i64,i64
"""Murshidabad""",34970,111585,159840
"""South 24 Parganas""",27745,121398,145339
"""North 24 Parganas""",23194,110944,147672
"""Uttar Dinajpur""",20229,40426,73513
"""Paschim Medinipur""",15095,80435,92761
…,…,…,…
"""Dakshin Dinajpur""",5882,24184,27027
"""Jalpaiguri""",5788,36039,54352
"""Kolkata""",5295,30962,73393


In [24]:
top_districts_state = top_districts_state.with_columns(
    (
        (pl.col("bio517_sum") + pl.col("bio17_sum"))
        / pl.col("enrol_sum")
    ).alias("biometric_per_enrol")
)

top_districts_state


district,enrol_sum,bio517_sum,bio17_sum,biometric_per_enrol
str,i64,i64,i64,f64
"""Murshidabad""",34970,111585,159840,7.761653
"""South 24 Parganas""",27745,121398,145339,9.613876
"""North 24 Parganas""",23194,110944,147672,11.150125
"""Uttar Dinajpur""",20229,40426,73513,5.632458
"""Paschim Medinipur""",15095,80435,92761,11.473733
…,…,…,…,…
"""Dakshin Dinajpur""",5882,24184,27027,8.706392
"""Jalpaiguri""",5788,36039,54352,15.616966
"""Kolkata""",5295,30962,73393,19.708215


In [27]:
top_districts_state.write_csv("west_bengal_top_districts.csv")


In [28]:
state_name = "Maharashtra"

top_districts_state = (
    df_all
    .filter(pl.col("state") == state_name)
    .group_by("district")
    .agg([
        pl.col("enrol_total").sum().alias("enrol_sum"),
        pl.col("bioage517").sum().alias("bio517_sum"),
        pl.col("bioage17").sum().alias("bio17_sum"),
    ])
    .sort("enrol_sum", descending=True)
    .head(20)
)

top_districts_state


district,enrol_sum,bio517_sum,bio17_sum
str,i64,i64,i64
"""Thane""",31803,245906,442903
"""Pune""",25014,329577,373112
"""Nashik""",16881,246089,421822
"""Mumbai Suburban""",14426,120606,90043
"""Aurangabad""",11993,126826,230945
…,…,…,…
"""Kolhapur""",6404,132531,170903
"""Amravati""",6026,119460,267732
"""Latur""",5971,84750,193647


In [29]:
top_districts_state = top_districts_state.with_columns(
    (
        (pl.col("bio517_sum") + pl.col("bio17_sum"))
        / pl.col("enrol_sum")
    ).alias("biometric_per_enrol")
)

top_districts_state


district,enrol_sum,bio517_sum,bio17_sum,biometric_per_enrol
str,i64,i64,i64,f64
"""Thane""",31803,245906,442903,21.658617
"""Pune""",25014,329577,373112,28.091829
"""Nashik""",16881,246089,421822,39.565843
"""Mumbai Suburban""",14426,120606,90043,14.602038
"""Aurangabad""",11993,126826,230945,29.831652
…,…,…,…,…
"""Kolhapur""",6404,132531,170903,47.381949
"""Amravati""",6026,119460,267732,64.253568
"""Latur""",5971,84750,193647,46.624853


In [30]:
top_districts_state.write_csv("maharashtra_top_districts.csv")


In [31]:
best_district = top_districts_state.sort("enrol_sum", descending=True).head(1)
best_district


district,enrol_sum,bio517_sum,bio17_sum,biometric_per_enrol
str,i64,i64,i64,f64
"""Thane""",31803,245906,442903,21.658617


In [32]:
# if you already added biometric_per_enrol in the previous step
top_by_biometric_ratio = (
    top_districts_state
    .sort("biometric_per_enrol", descending=True)
    .head(10)
)

top_by_biometric_ratio


district,enrol_sum,bio517_sum,bio17_sum,biometric_per_enrol
str,i64,i64,i64,f64
"""Yavatmal""",5798,106863,267919,64.639876
"""Amravati""",6026,119460,267732,64.253568
"""Kolhapur""",6404,132531,170903,47.381949
"""Ahmadnagar""",8776,186854,224512,46.873974
"""Latur""",5971,84750,193647,46.624853
"""Jalgaon""",10519,169106,304509,45.024717
"""Nanded""",9140,145570,249903,43.268381
"""Mumbai""",10874,136457,324304,42.372724
"""Nashik""",16881,246089,421822,39.565843


In [33]:
district_name = "Murshidabad"

murshidabad_trend = (
    df_all
    .filter(
        (pl.col("state") == "West Bengal")
        & (pl.col("district") == district_name)
    )
    .group_by("date")
    .agg([
        pl.col("enrol_total").sum().alias("enrol_sum"),
        pl.col("bioage517").sum().alias("bio517_sum"),
        pl.col("bioage17").sum().alias("bio17_sum"),
    ])
    .sort("date")
)

murshidabad_trend


date,enrol_sum,bio517_sum,bio17_sum
date,i64,i64,i64
2025-03-01,0,11142,18296
2025-04-01,0,8624,14934
2025-05-01,0,12291,15617
2025-06-01,0,11448,17891
2025-07-01,0,19913,25138
…,…,…,…
2025-12-25,68,720,696
2025-12-26,376,606,696
2025-12-27,358,730,628


In [34]:
district_ratio_all = (
    df_all
    .group_by(["state", "district"])
    .agg([
        pl.col("enrol_total").sum().alias("enrol_sum"),
        pl.col("bioage517").sum().alias("bio517_sum"),
        pl.col("bioage17").sum().alias("bio17_sum"),
    ])
    .with_columns(
        (
            (pl.col("bio517_sum") + pl.col("bio17_sum"))
            / pl.col("enrol_sum")
        ).alias("biometric_per_enrol")
    )
    .sort("biometric_per_enrol", descending=True)
    .head(20)
)

district_ratio_all


state,district,enrol_sum,bio517_sum,bio17_sum,biometric_per_enrol
str,str,i64,i64,i64,f64
"""Uttar Pradesh""","""Bagpat""",0,2,6,inf
"""Jammu and Kashmir""","""?""",0,0,1,inf
"""Jammu & Kashmir""","""Rajauri""",0,2,10,inf
"""Uttar Pradesh""","""Chandauli *""",0,0,1,inf
"""Karnataka""","""Bengaluru Rural""",0,5,5,inf
…,…,…,…,…,…
"""WEST BENGAL""","""HOOGHLY""",0,1,15,inf
"""Sikkim""","""Namchi""",0,0,5,inf
"""Rajasthan""","""Balotra""",0,0,8,inf


In [35]:
top_districts_state.write_csv("west_bengal_summary_final.csv")
district_ratio_all.write_csv("district_ratio_all_final.csv")
